# 第11章 接口: 从协议到抽象基类

## 11.1 Python文化中的接口和协议

示例 11-2 使用特性实现x和y

In [1]:
class Vector2d:

    def __init__(self, x, y) -> None:
        self.__x = x
        self.__y = y
    
    @property
    def x(self):
        return self.__x
    
    @property
    def y(self):
        return self.__y
    
    def __iter__(self):
        return (i for i in (self.x, self.y))

In [4]:
v = Vector2d(2, 3)
v.x, v.y,  [x for x in v]

(2, 3, [2, 3])

## 11.2 Python喜欢序列

Python会特殊对待看起来像是序列的对象。Python 中的迭代是鸭子类型的一种极端形式： 为了迭代对象, 解释器会尝试调用两种不同的方法

## 11.3 使用猴子补丁在运行时实现协议

In [2]:
from random import shuffle
l = list(range(10))
shuffle(l)
l

[6, 0, 1, 2, 8, 7, 5, 4, 3, 9]

示例 11-6 为FrenchDeck打猴子补丁, 把它变成可变的, 让random.shuffle 函数可以处理

In [17]:
import types
import collections

# 命名元组: 和C中的结构体很相似
Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list('JQKA')
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

def set_card(deck, pos, card):
    """设置"""
    deck._cards[pos] = card

class FrenchDeck2(FrenchDeck): ...
deck = FrenchDeck2()
deck.__class__.__setitem__ = set_card
shuffle(deck)
deck[:5]

[Card(rank='2', suit='spades'),
 Card(rank='Q', suit='hearts'),
 Card(rank='9', suit='hearts'),
 Card(rank='9', suit='clubs'),
 Card(rank='K', suit='hearts')]

<mark />鸭子类型: 对象的类型无关紧要, 只要实现了特定的协议即可</mark>

## 11.4 Alex martelli 的水禽

In [20]:
class Struggle:
    def __len__(self): return 23
class Struggle2: ...
from collections import abc
[isinstance(cls(), abc.Sized) for cls in (Struggle, Struggle2)]

[True, False]

## 11.5 定义抽象基类的子类

In [25]:
import collections
import collections.abc

Card = collections.namedtuple('Card', ['rank', 'suit'])

class FrenchDeck2(collections.abc.MutableSequence):
    ranks = [str(n) for n in range(2, 11)] + list("JQKA")
    suits = 'spades diamonds clubs hearts'.split()

    def __init__(self) -> None:
        self._cards = [Card(rank, suit) for suit in self.suits for rank in self.ranks]
    
    def __len__(self):
        return len(self._cards)
    
    def __getitem__(self, pos):
        return self._cards[pos]
    
    def __setitem__(self, pos, val):
        self._cards[pos] = val
    
    def __delitem__(self, pos):
        del self._cards[pos]
    
    def insert(self, index: int, value):
        self._cards.insert(index, value)

In [26]:
deck = FrenchDeck2()

## 11.6 标准库中的抽象基类

### 11.6.1 collections.abc模块中的抽象基类

In [27]:
import collections

In [38]:
abc_base_cls = sorted([ _ for _ in collections.abc.__dir__() if '_' not in _ ])
[abc_base_cls[i:i+4] for i in range(0, len(abc_base_cls), 4)]

[['AsyncGenerator', 'AsyncIterable', 'AsyncIterator', 'Awaitable'],
 ['ByteString', 'Callable', 'Collection', 'Container'],
 ['Coroutine', 'Generator', 'Hashable', 'ItemsView'],
 ['Iterable', 'Iterator', 'KeysView', 'Mapping'],
 ['MappingView', 'MutableMapping', 'MutableSequence', 'MutableSet'],
 ['Reversible', 'Sequence', 'Set', 'Sized'],
 ['ValuesView']]

### 11.6.2 抽象基类的数字塔

In [87]:
import numbers
import numpy as np
from tabulate import tabulate

COLORS = {
    'default': '\033[0m',
    'green': '\033[92m',
    'red': '\033[91m',
    'yellow': '\033[93m',
    'blue': '\033[94m'
}

# 数据准备
nums = [1, 1.0, complex(1, -1), True, "1", np.int64(0), np.float32(1.0), np.complex64(1), np.uint(12)]
objtype = [numbers.Number, numbers.Complex, numbers.Integral, numbers.Real, numbers.Rational]
headers = ['Type'] + [str(obj).split()[-1].split('>')[0].replace('\'', '') for obj in objtype]

# 创建数据行
rows = []
for num in nums:
    type_name = f"{type(num).__name__}"
    row = [COLORS['blue'] + type_name + COLORS['default']]
    row += [f"{COLORS['green']}✓{COLORS['default']}" if isinstance(num, obj) else f"{COLORS['red']}×{COLORS['default']}" for obj in objtype]
    rows.append(row)
# 打印表格
print(tabulate(rows, headers=headers, tablefmt='grid', stralign='center'))

+-----------+------------------+-------------------+--------------------+----------------+--------------------+
|   Type    |  numbers.Number  |  numbers.Complex  |  numbers.Integral  |  numbers.Real  |  numbers.Rational  |
+===========+==================+===================+====================+================+====================+
|    int    |        ✓         |         ✓         |         ✓          |       ✓        |         ✓          |
+-----------+------------------+-------------------+--------------------+----------------+--------------------+
|   float   |        ✓         |         ✓         |         ×          |       ✓        |         ×          |
+-----------+------------------+-------------------+--------------------+----------------+--------------------+
|  complex  |        ✓         |         ✓         |         ×          |       ×        |         ×          |
+-----------+------------------+-------------------+--------------------+----------------+--------------

## 11.7 定义并使用一个抽象基类

示例 11-9 Tombola是抽象基类, 有两个抽象方法和两个具体的方法

In [88]:
import abc

class Tombola(abc.ABC):

    @abc.abstractmethod
    def load(self, iterable):
        """从可迭代对象中添加元素"""
    @abc.abstractmethod
    def pick(self): 
        """随机删除元素, 然后将其返回。如果实例为空, 这个方法应该抛出`LookupError`"""
    def loaded(self):
        """如果至少有一个元素,返回`True`, 否则返回`False`。 """
        return bool(self.inspect())
    def inspect(self):
        """返回一个有序数组,由当前元素构成"""
        items = []
        while True:
            try:
                items.append(self.pick())
            except LookupError:
                break
        self.load(items)
        
        return tuple(sorted(items))

In [89]:
import random

class BingoCage(Tombola):

    def __init__(self, items) -> None:
        self._randomizer = random.SystemRandom()
        self._items = []
        self.load(items)
    
    def load(self, items):
        self._items.extend(items)
        self._randomizer.shuffle(self._items)
    
    def pick(self):
        try:
            return self._items.pop()
        except IndexError:
            raise LookupError('pick from empty BingoCage')
    
    def __call__(self):
        self.pick()

In [90]:
class LotteryBlower(Tombola):

    def __init__(self, iterable) -> None:
        self._balls = list(iterable)
    
    def load(self, iterable):
        self._balls.extend(iterable)

    def pick(self):
        try:
            pos = random.randrange(len(self._balls))
        except ValueError:
            raise LookupError('pick from empty LotteryBlower')
        return self._balls.pop(pos)
    
    def loaded(self):
        return bool(self._balls)

    def inspect(self):
        return tuple(sorted(self._balls))

实例 11-14 TomboList 是 Tombola 的虚拟子类

In [93]:
@Tombola.register
class TomboList(list):

    def pick(self):
        if self:
            pos = random.randrange(len(self))
            return self.pop(pos)
        else:
            raise LookupError('pop from empty TomboList')
    
    load = list.extend

    def loaded(self):
        return bool(self)
    
    def inspect(self):
        return tuple(sorted(self))

In [94]:
issubclass(TomboList, Tombola)

True

In [95]:
t = TomboList(range(100))
isinstance(t, Tombola)

True

In [96]:
TomboList.__mro__

(__main__.TomboList, list, object)

## 11.8 Tombola子类的测试方法

## 11.9 Python使用register的方式

In [116]:
class A(abc.ABC):
    pass

class B:
    pass

class C:
    pass

A.register(B)
issubclass(B, A), issubclass(C, A)

(True, False)

## 11.10 鹅的行为有可能像鸭子

In [102]:
class Struggle:
    def __len__(self): return 23

In [103]:
from collections import abc

In [104]:
isinstance(Struggle(), abc.Sized)

True

In [105]:
issubclass(Struggle, abc.Sized)

True

In [122]:
import abc

class A(abc.ABC):
    __slots__ = ()
    
    @abc.abstractmethod
    def __hello__(self): ...

    @classmethod
    def __subclasshook__(cls, C):
        if cls is A:
            if any("__hello__" in B.__dict__ for B in C.__mro__):
                return True
        return NotImplemented

class B:
    def __hello__(self):
        return "Hello World!"

print(issubclass(B, A))  # 输出: True


True


## 11.11 本章小结

<img src="./images/第11章总结1.jpg" width="70%">
<img src="./images/第11章总结2.jpg" width="70%">
<img src="./images/第11章总结3.jpg" width="70%">